# 1. Converting our PyTorch file into ONNX

First, we have to convert our PyTorch (.pt) file into a ONNX file, so that it is in a format ready to be compiled onto our hardware accelerator [Raspberry Pi AI Hat+](https://www.raspberrypi.com/products/ai-hat/). This device features a 13/26 TOPS module that can be combined with the Raspberry Pi 5.

In [ ]:
!pip install ultralytics
!pip install onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.0/683.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Converting and exporting our PyTorch file

In [ ]:
from ultralytics import YOLO


model = YOLO("/content/drive/MyDrive/yolo11s_kitti_runs/yolo11s_kitti/weights/best.pt")

model.export(
    format="onnx",
    imgsz=640,
    opset=11,
    nms=False,
    dynamic=False
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (AMD EPYC 7B12)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolo11s_kitti_runs/yolo11s_kitti/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (18.3 MB)
requirements: Ultralytics requirement ['onnxslim>=0.1.71'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 10 packages in 231ms
Prepared 2 packages in 44ms
Installed 2 packages in 8ms
 + colorama==0.4.6
 + onnxslim==0.1.77

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or re

'/content/drive/MyDrive/yolo11s_kitti_runs/yolo11s_kitti/weights/best.onnx'

Testing the validity of our model to ensure that our conversion succesfully worked

In [ ]:
import onnx
import onnxruntime as ort
import torch

# Load the ONNX model
onnx_model = onnx.load("/content/drive/MyDrive/yolo11s_kitti_runs/yolo11s_kitti/weights/best.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")

# Test the ONNX model with ONNX Runtime
dummy_input = torch.randn(1, 3, 640, 640).numpy()
ort_session = ort.InferenceSession("/content/drive/MyDrive/yolo11s_kitti_runs/yolo11s_kitti/weights/best.onnx")
outputs = ort_session.run(None, {"images": dummy_input})
print(outputs[0])

ONNX model is valid!
[[[     5.9055       15.73      18.201 ...      559.28      575.53       614.7]
  [     6.2303      4.7434      4.1057 ...      625.32      624.54      620.23]
  [     11.851      29.387      30.391 ...      143.49      122.01      51.392]
  ...
  [ 0.00025582   0.0001142  3.3677e-05 ...  1.5706e-05   1.651e-05  7.4655e-05]
  [ 4.2319e-06  5.9605e-07  4.7684e-07 ...  5.6326e-06  2.4438e-06  5.4836e-06]
  [  0.0019592  0.00051224  0.00011963 ...  2.4468e-05  1.3232e-05  5.8442e-05]]]


# 2. Downloading Hailo's Dataflow Compiler
This tool is to convert our ONNX model into it's final format to be used on the Hailo 8.

There's three ways to do this:
1. Use the Dataflow Compiler (DFC)
2. Use the [Model Zoo](https://github.com/hailo-ai/hailo_model_zoo) Python Integration
3. Use the [AI SW Docker Suite](https://hailo.ai/developer-zone/software-downloads/)

The DFC allows us to convert models that aren't already precompiled in the model-zoo, and allows us to retrain/convert custom models that have similar architectures are those already in the model zoo.

 To use this, we will have to convert our generated ONNX file into the HEF (Hailo Executable File) format used by the Hailo 8.

 The steps are as follows:


1.   [Parsing](https://hailo.ai/developer-zone/documentation/v3-29-0/?sp_referrer=tutorials_notebooks%2Fnotebooks%2FDFC_1_Parsing_Tutorial.html) the model
2.   [Optimization](https://hailo.ai/developer-zone/documentation/v3-29-0/?sp_referrer=tutorials_notebooks%2Fnotebooks%2FDFC_2_Model_Optimization_Tutorial.html&highlight=layers)
3. [Compilation](https://hailo.ai/developer-zone/documentation/v3-29-0/?page=tutorials_notebooks%2Fnotebooks%2FDFC_3_Compilation_Tutorial.html)
4. [Running inference](https://hailo.ai/developer-zone/documentation/v3-29-0/file/?page=tutorials_notebooks%2Fnotebooks%2FDFC_4_Inference_Tutorial.html)


Installing all necessary site packages

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-dev python3-distutils python3-tk libfuse2 graphviz libgraphviz-dev

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/mai

Installing the virtualenv package

In [ ]:
!pip install --upgrade pip virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 152.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 42.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


Creating a venv

In [ ]:
!virtualenv my_env

created virtual environment CPython3.12.12.final.0-64 in 253ms
  creator CPython3Posix(dest=/content/my_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.3
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Installing the DFC inside our venv, note that we have to install it using our python version within the virtual environment

In [ ]:
#Installing the WHL file for Hailo DFC
#Download the .whl file from https://hailo.ai/developer-zone/software-downloads/
#Should be Software Package: AI Software Suite -> Dataflow Compiler -> x86 -> Linux -> Python 3.10
!my_env/bin/pip install /content/drive/MyDrive/hailo_dataflow_compiler-3.33.0-py3-none-linux_x86_64.whl

Processing ./drive/MyDrive/hailo_dataflow_compiler-3.33.0-py3-none-linux_x86_64.whl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 125.2 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
INFO: pip is looking at multiple versions of matplotlib to determine which version is compatible with other requirements. This could 

Making sure it's properly installed

In [ ]:
!my_env/bin/hailo --version

[info] Current Time: 13:36:50, 11/30/25
[info] CPU: Architecture: x86_64, Model: Intel(R) Xeon(R) CPU @ 2.20GHz, Number Of Cores: 12, Utilization: 0.3%
[info] Memory: Total: 167GB, Available: 163GB
[info] System info: OS: Linux, Kernel: 6.6.105+
[info] Hailo DFC Version: 3.33.0
[info] HailoRT Version: Not Installed
[info] PCIe: No Hailo PCIe device was found
[info] Running `hailo --version`
Hailo Dataflow Compiler v3.33.0


### 2a. Parsing our ONNX file

#### Identifying appropriate end node names

The first step the parsing process is to tell the tool which layers/nodes to expect the output from, aka (end nodes). Once we identify which end nodes are the correct ones using Netron, we can continue with the first step of our compilation, parsing.

Now, once we have identified our end nodes, (you can click on them in netron to find the names, use node name).

In [ ]:
with open("translate_model.py", "w") as f:
    f.write("""
from hailo_sdk_client import ClientRunner

# Define the ONNX model path and configuration
onnx_path = "/content/drive/MyDrive/yolo11s_kitti_runs/yolo11s_kitti/weights/best.onnx"
onnx_model_name = "yolo11s_kitti"
chosen_hw_arch = "hailo8l"  # Specify the target hardware architecture

# Initialize the ClientRunner
runner = ClientRunner(hw_arch=chosen_hw_arch)

# Use the recommended end node names for translation for yolo
end_node_names = [
    "/model.23/cv2.0/cv2.0.2/Conv",
    "/model.23/cv3.0/cv3.0.2/Conv",
    "/model.23/cv2.1/cv2.1.2/Conv",
    "/model.23/cv3.1/cv3.1.2/Conv",
    "/model.23/cv2.2/cv2.2.2/Conv",
    "/model.23/cv3.2/cv3.2.2/Conv"
]
net_input_shapes={"images": [1, 3, 640, 640]}  # Adjust input shapes if needed

try:
    # Translate the ONNX model to Hailo's format
    hn, npz = runner.translate_onnx_model(
        onnx_path,
        onnx_model_name,
        end_node_names=end_node_names,
        net_input_shapes=net_input_shapes,  # Adjust input shapes if needed
    )
    print("Model translation successful.")
except Exception as e:
    print(f"Error during model translation: {e}")
    raise

# Save the Hailo model HAR file
hailo_model_har_name = f"{onnx_model_name}_hailo_model.har"
try:
    runner.save_har(hailo_model_har_name)
    print(f"HAR file saved as: {hailo_model_har_name}")
except Exception as e:
    print(f"Error saving HAR file: {e}")
    """)

Running our script with the venv python

In [ ]:
!my_env/bin/python translate_model.py

[info] Translation started on ONNX model yolo11s_kitti
[info] Restored ONNX model yolo11s_kitti (completion time: 00:00:00.31)
[info] Extracted ONNXRuntime meta-data for Hailo model (completion time: 00:00:01.69)
[info] NMS structure of yolov8 (or equivalent architecture) was detected.
[info] In order to use HailoRT post-processing capabilities, these end node names should be used: /model.23/cv3.0/cv3.0.2/Conv /model.23/cv2.0/cv2.0.2/Conv /model.23/cv3.1/cv3.1.2/Conv /model.23/cv2.1/cv2.1.2/Conv /model.23/cv2.2/cv2.2.2/Conv /model.23/cv3.2/cv3.2.2/Conv.
[info] Start nodes mapped from original model: 'images': 'yolo11s_kitti/input_layer1'.
[info] End nodes mapped from original model: '/model.23/cv2.0/cv2.0.2/Conv', '/model.23/cv3.0/cv3.0.2/Conv', '/model.23/cv2.1/cv2.1.2/Conv', '/model.23/cv3.1/cv3.1.2/Conv', '/model.23/cv2.2/cv2.2.2/Conv', '/model.23/cv3.2/cv3.2.2/Conv'.
[info] Translation completed on ONNX model yolo11s_kitti (completion time: 00:00:03.85)
Model translation successful

### 4b. Optimizing our HAR file

Downloading our file

Since Hailo has their own renaming process, we have to find the new names of our end nodes. Below is a dictionary of layers and operations, find the end node names identified by output_layers_order.

In [ ]:
with open("inspect_dict.py", "w") as f:
    f.write("""
from hailo_sdk_client import ClientRunner

# Load the HAR file
har_path = "/content/yolo11s_kitti_hailo_model.har"

runner = ClientRunner(har=har_path)

from pprint import pprint

try:
    # Access the HailoNet as an OrderedDict
    hn_dict = runner.get_hn()  # Or use runner._hn if get_hn() is unavailable
    print("Inspecting layers from HailoNet (OrderedDict):")

    # Pretty-print each layer
    for key, value in hn_dict.items():
        print(f"Key: {key}")
        pprint(value)
        print("\\n" + "="*80 + "\\n")  # Add a separator between layers for clarity

except Exception as e:
    print(f"Error while inspecting hn_dict: {e}")
""")

In [ ]:
!my_env/bin/python inspect_dict.py

Inspecting layers from HailoNet (OrderedDict):
Key: name
'yolo11s_kitti'


Key: net_params
OrderedDict([('version', '1.0'),
             ('stage', 'HN'),
             ('clusters_placement', [[]]),
             ('clusters_to_skip', []),
             ('output_layers_order',
              ['yolo11s_kitti/conv51',
               'yolo11s_kitti/conv54',
               'yolo11s_kitti/conv62',
               'yolo11s_kitti/conv65',
               'yolo11s_kitti/conv77',
               'yolo11s_kitti/conv80']),
             ('is_transformer', True),
             ('transposed_net', False),
             ('net_scopes', ['yolo11s_kitti']),
             ('lora_adapters', [])])


Key: layers
OrderedDict([('yolo11s_kitti/input_layer1',
              OrderedDict([('type', 'input_layer'),
                           ('input', []),
                           ('output', ['yolo11s_kitti/conv1']),
                           ('input_shapes', [[-1, 640, 640, 3]]),
                           ('output_shapes', 

Next, generating calibration data to optimize our model, this helps the DFC optimize the network model and parameters using a pre-processed calibration set. This only needs to be done once.


In [ ]:
import numpy as np
from PIL import Image
import os
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive/', force_remount=True)

# Paths to directories and files
image_dir = '/content/drive/MyDrive/calibration_dataset_kitti'
output_dir = '/content/drive/MyDrive/kitti_yolo'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# File paths for saving calibration data
calibration_data_path = os.path.join(output_dir, "calibration_data.npy")
processed_data_path = os.path.join(output_dir, "processed_calibration_data.npy")

# Initialize an empty list for calibration data
calib_data = []

# Process all image files in the directory
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        img = Image.open(img_path).resize((640, 640))  # Resize to desired dimensions
        img_array = np.array(img) / 255.0  # Normalize to [0, 1]
        calib_data.append(img_array)

# Convert the calibration data to a NumPy array
calib_data = np.array(calib_data)

# Save the normalized calibration data
np.save(calibration_data_path, calib_data)
print(f"Normalized calibration dataset saved with shape: {calib_data.shape} to {calibration_data_path}")

# Scale the normalized data back to [0, 255]
processed_calibration_data = calib_data * 255.0

# Save the processed calibration data
np.save(processed_data_path, processed_calibration_data)
print(f"Processed calibration dataset saved with shape: {processed_calibration_data.shape} to {processed_data_path}")


Mounted at /content/drive/
Normalized calibration dataset saved with shape: (1360, 640, 640, 3) to /content/drive/MyDrive/kitti_yolo/calibration_data.npy
Processed calibration dataset saved with shape: (1360, 640, 640, 3) to /content/drive/MyDrive/kitti_yolo/processed_calibration_data.npy


Creating a NMS layer config to properly match our modified model architecture

In [ ]:
import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive/', force_remount=True)

# Updated NMS layer configuration dictionary
nms_layer_config = {
    "nms_scores_th": 0.2,
    "nms_iou_th": 0.7,
    "image_dims": [640, 640],
    "max_proposals_per_class": 100,
    "classes": 8,
    "regression_length": 16,
    "background_removal": False,
    "background_removal_index": 0,
    "bbox_decoders": [
        {
            "name": "scale_stride_8",
            "stride": 8,
            "reg_layer": "yolo11s_kitti/conv51",
            "cls_layer": "yolo11s_kitti/conv54"
        },
        {
            "name": "scale_stride_16",
            "stride": 16,
            "reg_layer": "yolo11s_kitti/conv62",
            "cls_layer": "yolo11s_kitti/conv65"
        },
        {
            "name": "scale_stride_32",
            "stride": 32,
            "reg_layer": "yolo11s_kitti/conv77",
            "cls_layer": "yolo11s_kitti/conv80"
        }
    ]
}

# Path to save the updated JSON configuration
output_dir = "/content/drive/MyDrive"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
output_path = os.path.join(output_dir, "nms_layer_config.json")

# Save the updated configuration as a JSON file
with open(output_path, "w") as json_file:
    json.dump(nms_layer_config, json_file, indent=4)

print(f"NMS layer configuration saved to {output_path}")

Mounted at /content/drive/
NMS layer configuration saved to /content/drive/MyDrive/nms_layer_config.json


Now writing a script to optimize our model, you can find sample .alls files in the Hailo model zoo.

In [ ]:
with open("optimize_model.py", "w") as f:
    f.write("""
import os
from hailo_sdk_client import ClientRunner

# Define your model's HAR file name
model_name = "yolo11s_kitti"
hailo_model_har_name = f"{model_name}_hailo_model.har"

# Ensure the HAR file exists
assert os.path.isfile(hailo_model_har_name), "Please provide a valid path for the HAR file"

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=hailo_model_har_name)

# Define the model script to add a normalization layer
# Normalization for [0, 1] range
alls = \"\"\"
normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])
change_output_activation(yolo11s_kitti/conv54, sigmoid)
change_output_activation(yolo11s_kitti/conv65, sigmoid)
change_output_activation(yolo11s_kitti/conv80, sigmoid)
nms_postprocess("/content/drive/MyDrive/nms_layer_config.json", meta_arch=yolov8, engine=cpu)
\"\"\"

# Load the model script into the ClientRunner
runner.load_model_script(alls)

# Define a calibration dataset
# Replace 'calib_dataset' with the actual dataset you're using for calibration
# For example, if it's a directory of images, prepare the dataset accordingly
calib_dataset = "/content/drive/MyDrive/kitti_yolo/processed_calibration_data.npy"

# Perform optimization with the calibration dataset
runner.optimize(calib_dataset)

# Save the optimized model to a new Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

print(f"Quantized HAR file saved to: {quantized_model_har_path}")
    """)

Running it with venv python

In [ ]:
!my_env/bin/python optimize_model.py

[info] Loading model script commands to yolo11s_kitti from string
[info] Found model with 3 input channels, using real RGB images for calibration instead of sampling random data.
[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn
[info] MatmulDecompose skipped
[info] Starting Mixed Precision
[info] Model Optimization Algorithm Mixed Precision is done (completion time is 00:00:00.66)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration
Calibration: 100% 64/64 [01:10<00:00,  1.10s/entries]
[info] Model Optimization Algorithm Statistics Collector is done (completion time is 00:01:12.78)
[i

### 4c. Running final compilation into a HEF file

Writing a local script to do the conversion

In [ ]:
with open("compile_model.py", "w") as f:
    f.write("""
from hailo_sdk_client import ClientRunner

# Define the quantized model HAR file
model_name = "yolo11s_kitti"
quantized_model_har_path = f"{model_name}_quantized_model.har"


# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=quantized_model_har_path)
print("[info] ClientRunner initialized successfully.")

# Compile the model
try:
    hef = runner.compile()
    print("[info] Compilation completed successfully.")
except Exception as e:
    print(f"[error] Failed to compile the model: {e}")
    raise
file_name = f"{model_name}.hef"
with open(file_name, "wb") as f:
    f.write(hef)
    """)

Running it with venv python

In [ ]:
!my_env/bin/python compile_model.py

[info] ClientRunner initialized successfully.
[info] To achieve optimal performance, set the compiler_optimization_level to "max" by adding performance_param(compiler_optimization_level=max) to the model script. Note that this may increase compilation time.
[info] Loading network parameters
[info] Starting Hailo allocation and compilation flow
[info] Adding an output layer after conv51
[info] Adding an output layer after conv54
[info] Adding an output layer after conv62
[info] Adding an output layer after conv65
[info] Adding an output layer after conv77
[info] Adding an output layer after conv80
[info] Building optimization options for network layers...
[info] Successfully built optimization options - 10s 413ms
[info] Trying to compile the network in a single context
[info] Single context flow failed: Recoverable single context error
[info] Building optimization options for network layers...
[info] Successfully built optimization options - 17s 901ms
[info] Using Multi-context flow
[in

In [ ]:
!mv "/content/drive/MyDrive/yolo11s_kitti.hef" "/content/drive/MyDrive/hailo8l_hef/"

